In [1]:
import findspark

findspark.init()

ModuleNotFoundError: No module named 'findspark'

In [ ]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
data = spark.read.format("libsvm").load("iris-train.txt")

In [ ]:
# Split the training data into train and test for parameter tuning
splits = data.randomSplit([0.6, 0.4], 1)
train = splits[0]
test = splits[1]

bestAccuracy = 0
for l in [[4, 3], [4, 5, 3], [4, 5, 4, 3], [4, 10, 7, 5, 3]]:
    for i in range(10,51,10):
        for s in [64, 128, 256]:
            # specify layers for the neural network:
            # input layer of size numfeatures, optional intermediate layers
            # and output of size numclasses
            layers = l

            # create the trainer and set its parameters
            trainer = MultilayerPerceptronClassifier(maxIter=i, layers=layers, blockSize=s, seed=1)

            # train the model
            model = trainer.fit(train)
            
            # compute accuracy on the test set
            result = model.transform(test)
            predictionAndLabels = result.select("prediction", "label")
            evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
            accuracy = evaluator.evaluate(predictionAndLabels)
            
            if accuracy > bestAccuracy:
                bestAccuracy = accuracy
                bestL = l
                bestI = i
                bestS = s

In [ ]:
print("Best accuracy = " + str(bestAccuracy) + " obtained with parameters layers = " + str(bestL) + " maxIter=" + str(bestI) + " and blockSize=" + str(bestS))

In [ ]:
result.show(result.count(), truncate=25)